In [59]:
import pandas as pd
df = pd.read_csv('scores.csv')
import sqlite3
from sqlite3 import Error
rowCols = ['Percent Black', 'Percent White', 'Percent Asian', 'Percent Hispanic']
neededCols = [rowCols[:1][0], 'Average Score (SAT Math)']
sat_data = []
df.shape

(435, 22)

In [60]:
for col in neededCols:
    df = df[pd.notnull(df[col])]
df.shape

(375, 22)

In [61]:
def extractValues(row):
    other = 100
    rowData = {}
    for col in rowCols:
            rowData[col] = float(row[col].strip('%'))
            other -= float(row[col].strip('%'))
    rowData['name'] = row['School Name']
    rowData['math_score'] = row['Average Score (SAT Math)']
    rowData['reading_score'] = row['Average Score (SAT Reading)']     
    rowData['writing_score'] = row['Average Score (SAT Writing)']
    if other < 0:
        other = 0
    sat_data.append((row['School Name'], rowData['Percent Black'], rowData['Percent Asian'], rowData['Percent White'], rowData['Percent Hispanic'], other, rowData['math_score'], rowData['reading_score'], rowData['writing_score']))
                
for index, (_, row) in enumerate(df.iterrows()):
    extractValues(row)
                        

In [62]:
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        for school in sat_data:
            add_school(conn, school)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        conn.commit()
        conn.close()
        
def add_school(conn, school):
    sql = """ 
    INSERT INTO schools(name,percent_black,percent_asian,percent_white,percent_hispanic,percent_other,math_score,reading_score,writing_score)
    VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    curr = conn.cursor()
    curr.execute(sql, school)
    return curr.lastrowid

create_connection('pythonsqlite.db')

2.6.0
